# やさしく学ぶディープラーニングがわかる数学の基本 
# Chapter5 ニューラルネットワークを実装してみよう

## Section2 アスペクト比判定ニューラルネットワーク

In [1]:
import numpy as np
import math

# 学習データの数
N = 1000

# （学習に再現性を持たせるためにシードを固定しています。本来は不要です）
np.random.seed(1)

# 適当な学習データと正解ラベルを生成
TX = (np.random.rand(N, 2) * 1000).astype(np.int32) + 1
TY = (TX.min(axis=1) / TX.max(axis=1) <= 0.2).astype(np.int)[np.newaxis].T #  行方向がaxis=0、列方向がaxis=1

### 標準化

学習データの平均を0、分散を1に揃えることで、パラメータの収束の速度を上げることができる

In [2]:
# 平均と標準偏差を計算
MU = TX.mean(axis=0) # 平均
SIGMA = TX.std(axis=0) # 標準偏差

# 標準化
def standardize(X):
        return (X - MU) / SIGMA

TX = standardize(TX)

## Section2 Step1. ニューラルネットワークの構造

In [3]:
# 重みとバイアス
W1 = np.random.randn(2, 2) # 第1層重み（2x2行列）
W2 = np.random.randn(2, 2) # 第2層重み（2x2行列）
W3 = np.random.randn(1, 2) # 第3層重み（1x2行列）
b1 = np.random.randn(2) # 第1層バイアス（2x1行列）
b2 = np.random.randn(2) # 第2層バイアス（2x1行列）
b3 = np.random.randn(2) # 第3層バイアス（1x1行列）

## Section2 Step2. 順伝播

In [4]:
# シグモイド関数
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

In [5]:
# 順伝播
def forward(X0):
    Z1 = np.dot(X0, W1.T) + b1
    X1 = sigmoid(Z1)
    Z2 = np.dot(X1, W2.T) + b2
    X2 = sigmoid(Z2)
    Z3 = np.dot(X2, W3.T) + b3
    X3 = sigmoid(Z3)

    return Z1, X1, Z2, X2, Z3, X3

## Section2 Step3. 逆伝播

逆伝播の処理に必要なのは、シグモイド関数の微分、出力層のデルタ、隠れ層のデルタの3つ。それぞれ順番に実装していく

In [6]:
# シグモイド関数の微分
def dsigmoid(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)

In [7]:
# 出力層のデルタ
def delta_output(Z, Y):
    return (sigmoid(Z) - Y) * dsigmoid(Z)

In [8]:
# 隠れ層のデルタ
def delta_hidden(Z, D, W):
    return dsigmoid(Z) * np.dot(D, W)

In [9]:
# 逆伝播
def backward(Y, Z3, Z2, Z1):
    D3 = delta_output(Z3, Y)
    D2 = delta_hidden(Z2, D3, W3)
    D1 = delta_hidden(Z1, D2, W2)

    return D3, D2, D1

## Section2 Step4. 学習

In [10]:
# 学習率
ETA = 0.001

In [11]:
# 目的関数の重みでの微分
def dweight(D, X):
    return np.dot(D.T, X)

In [12]:
# 目的関数のバイアスでの微分
def dbias(D):
    return D.sum(axis=0)

In [13]:
# パラメータの更新
def update_parameters(D3, X2, D2, X1, D1, X0):
    global W3, W2, W1, b3, b2, b1

    W3 = W3 - ETA * dweight(D3, X2)
    W2 = W2 - ETA * dweight(D2, X1)
    W1 = W1 - ETA * dweight(D1, X0)

    b3 = b3 - ETA * dbias(D3)
    b2 = b2 - ETA * dbias(D2)
    b1 = b1 - ETA * dbias(D1)

In [14]:
# 学習
def train(X, Y):
    # 順伝播
    Z1, X1, Z2, X2, Z3, X3 = forward(X)

    # 逆伝播
    D3, D2, D1 = backward(Y, Z3, Z2, Z1)

    # パラメータの更新
    update_parameters(D3, X2, D2, X1, D1, X)

In [15]:
# 繰り返し回数
EPOCH = 30000

In [16]:
# 予測
def predict(X):
    return forward(X)[-1]

# 目的関数
def E(Y, X):
    return 0.5 * ((Y - predict(X)) ** 2).sum()

## Section2 Step５. ミニバッチ法

In [17]:
# ミニバッチ数
BATCH = 100

for epoch in range(1, EPOCH + 1):
    # ミニバッチ学習用にランダムなインデックスを取得
    p = np.random.permutation(len(TX))

    # ミニバッチの数分だけデータを取り出して学習
    for i in range(math.ceil(len(TX) / BATCH)):
        indice = p[i*BATCH:(i+1)*BATCH]
        X0 = TX[indice]
        Y  = TY[indice]

        train(X0, Y)

    # ログを残す
    if epoch % 1000 == 0:
        log = '誤差 = {:8.4f} ({:5d}エポック目)'
        print(log.format(E(TY, TX), epoch))

ValueError: shapes (100,2) and (1,2) not aligned: 2 (dim 1) != 1 (dim 0)

（上記のセルでの実行だとエラーになるので、）ここまでのロジックをまとめて実行する

In [18]:
import numpy as np
import math

# 学習データの数
N = 1000

# (学習に再現性をもたせるために シードを固定しています。本来は不要です)
np.random.seed(1)

# 適当な学習データと正解ラベルを生成
TX = (np.random.rand(N, 2) * 1000).astype(np.int32) + 1
TY = (TX.min(axis=1) / TX.max(axis=1) <= 0.2).astype(np.int)[np.newaxis].T

# 平均と標準偏差を計算
MU = TX.mean(axis=0)
SIGMA = TX.std(axis=0)

# 標準化
def standardize(X):
    return (X - MU) / SIGMA

TX = standardize(TX)

# 重みとバイアス
W1 = np.random.randn(2, 2) # 第1層重み
W2 = np.random.randn(2, 2) # 第2層重み
W3 = np.random.randn(1, 2) # 第3層重み
b1 = np.random.randn(2)    # 第1層バイアス
b2 = np.random.randn(2)    # 第2層バイアス
b3 = np.random.randn(1)    # 第3層バイアス

# シグモイド関数
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

# 順伝播
def forward(X0):
    Z1 = np.dot(X0, W1.T) + b1
    X1 = sigmoid(Z1)
    Z2 = np.dot(X1, W2.T) + b2
    X2 = sigmoid(Z2)
    Z3 = np.dot(X2, W3.T) + b3
    X3 = sigmoid(Z3)

    return Z1, X1, Z2, X2, Z3, X3

# シグモイド関数の微分
def dsigmoid(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)

# 出力層のデルタ
def delta_output(Z, Y):
    return (sigmoid(Z) - Y) * dsigmoid(Z)

# 隠れ層のデルタ
def delta_hidden(Z, D, W):
    return dsigmoid(Z) * np.dot(D, W)

# 逆伝播
def backward(Y, Z3, Z2, Z1):
    D3 = delta_output(Z3, Y)
    D2 = delta_hidden(Z2, D3, W3)
    D1 = delta_hidden(Z1, D2, W2)

    return D3, D2, D1

# 学習率
ETA = 0.001

# 目的関数の重みでの微分
def dweight(D, X):
    return np.dot(D.T, X)

# 目的関数のバイアスでの微分
def dbias(D):
    return D.sum(axis=0)

# パラメータの更新
def update_parameters(D3, X2, D2, X1, D1, X0):
    global W3, W2, W1, b3, b2, b1

    W3 = W3 - ETA * dweight(D3, X2)
    W2 = W2 - ETA * dweight(D2, X1)
    W1 = W1 - ETA * dweight(D1, X0)

    b3 = b3 - ETA * dbias(D3)
    b2 = b2 - ETA * dbias(D2)
    b1 = b1 - ETA * dbias(D1)

# 学習
def train(X, Y):
    # 順伝播
    Z1, X1, Z2, X2, Z3, X3 = forward(X)

    # 逆伝播
    D3, D2, D1 = backward(Y, Z3, Z2, Z1)

    # パラメータの更新
    update_parameters(D3, X2, D2, X1, D1, X)

# 繰り返し回数
EPOCH = 30000

# 予測
def predict(X):
    return forward(X)[-1]

# 目的関数
def E(Y, X):
    return 0.5 * ((Y - predict(X)) ** 2).sum()

# ミニバッチ数
BATCH = 100

for epoch in range(1, EPOCH + 1):
    # ミニバッチ学習用にランダムなインデックスを取得
    p = np.random.permutation(len(TX))

    # ミニバッチの数分だけデータを取り出して学習
    for i in range(math.ceil(len(TX) / BATCH)):
        indice = p[i*BATCH:(i+1)*BATCH]
        X0 = TX[indice]
        Y  = TY[indice]

        train(X0, Y)

    # ログを残す
    if epoch % 1000 == 0:
        log = '誤差 = {:8.4f} ({:5d}エポック目)'
        print(log.format(E(TY, TX), epoch))

誤差 =  69.7705 ( 1000エポック目)
誤差 =  55.0522 ( 2000エポック目)
誤差 =  44.4299 ( 3000エポック目)
誤差 =  24.9272 ( 4000エポック目)
誤差 =  15.0983 ( 5000エポック目)
誤差 =  11.5577 ( 6000エポック目)
誤差 =   9.6375 ( 7000エポック目)
誤差 =   8.4023 ( 8000エポック目)
誤差 =   7.5279 ( 9000エポック目)
誤差 =   6.8681 (10000エポック目)
誤差 =   6.3472 (11000エポック目)
誤差 =   5.9225 (12000エポック目)
誤差 =   5.5672 (13000エポック目)
誤差 =   5.2649 (14000エポック目)
誤差 =   5.0039 (15000エポック目)
誤差 =   4.7761 (16000エポック目)
誤差 =   4.5755 (17000エポック目)
誤差 =   4.3975 (18000エポック目)
誤差 =   4.2384 (19000エポック目)
誤差 =   4.0956 (20000エポック目)
誤差 =   3.9667 (21000エポック目)
誤差 =   3.8496 (22000エポック目)
誤差 =   3.7430 (23000エポック目)
誤差 =   3.6453 (24000エポック目)
誤差 =   3.5555 (25000エポック目)
誤差 =   3.4725 (26000エポック目)
誤差 =   3.3955 (27000エポック目)
誤差 =   3.3238 (28000エポック目)
誤差 =   3.2566 (29000エポック目)
誤差 =   3.1936 (30000エポック目)


In [19]:
testX = standardize([
    [100, 100], # 正方形。細長くないはず
    [100, 10], # 細長いはず 
    [10, 100], # これも細長いはず 
    [80, 100], # これは細長くないはず            
])

predict(testX)

array([[0.00097628],
       [0.82436398],
       [0.94022858],
       [0.00173001]])

In [20]:
# 分類器
def classify(X):
    return (predict(X) > 0.8).astype(np.int)

# テストデータ生成
TEST_N = 1000
testX = (np.random.rand(TEST_N, 2) * 1000).astype(np.int32) + 1
testY = (testX.min(axis=1) / testX.max(axis=1) <= 0.2).astype(np.int)[np.newaxis].T

# 精度計算
accuracy = (classify(standardize(testX)) == testY).sum() / TEST_N
print('精度: {}%'.format(accuracy * 100))


精度: 98.4%
